# Navigation

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the first project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893).

### 1. Start the Environment

We begin by importing some necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [2]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Banana.app"`
- **Windows** (x86): `"path/to/Banana_Windows_x86/Banana.exe"`
- **Windows** (x86_64): `"path/to/Banana_Windows_x86_64/Banana.exe"`
- **Linux** (x86): `"path/to/Banana_Linux/Banana.x86"`
- **Linux** (x86_64): `"path/to/Banana_Linux/Banana.x86_64"`
- **Linux** (x86, headless): `"path/to/Banana_Linux_NoVis/Banana.x86"`
- **Linux** (x86_64, headless): `"path/to/Banana_Linux_NoVis/Banana.x86_64"`

For instance, if you are using a Mac, then you downloaded `Banana.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Banana.app")
```

In [3]:
env = UnityEnvironment(file_name="Banana_Windows_x86_64/Banana.exe")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [7]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

The simulation contains a single agent that navigates a large environment.  At each time step, it has four actions at its disposal:
- `0` - walk forward 
- `1` - walk backward
- `2` - turn left
- `3` - turn right

The state space has `37` dimensions and contains the agent's velocity, along with ray-based perception of objects around agent's forward direction.  A reward of `+1` is provided for collecting a yellow banana, and a reward of `-1` is provided for collecting a blue banana. 

Run the code cell below to print some information about the environment.

In [8]:
# reset the environment
env_info = env.reset(train_mode=False)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)

Number of agents: 1
Number of actions: 4
States look like: [0.         1.         0.         0.         0.16895212 0.
 1.         0.         0.         0.20073597 1.         0.
 0.         0.         0.12865657 0.         1.         0.
 0.         0.14938059 1.         0.         0.         0.
 0.58185619 0.         1.         0.         0.         0.16089135
 0.         1.         0.         0.         0.31775284 0.
 0.        ]
States have length: 37


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action (uniformly) at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

In [9]:
# env_info = env.reset(train_mode=False)[brain_name] # reset the environment
# state = env_info.vector_observations[0]            # get the current state
# score = 0                                          # initialize the score
# while True:
#     action = np.random.randint(action_size)        # select an action
#     env_info = env.step(action)[brain_name]        # send the action to the environment
#     next_state = env_info.vector_observations[0]   # get the next state
#     reward = env_info.rewards[0]                   # get the reward
#     done = env_info.local_done[0]                  # see if episode has finished
    
    
#     score += reward                                # update the score
#     state = next_state                             # roll over the state to next time step
#     if done:                                       # exit loop if episode finished
#         break
    
# print("Score: {}".format(score))

# env_info = env.reset(train_mode=True)[brain_name]


When finished, you can close the environment.### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [10]:
from dqn_agent import Agent
from collections import namedtuple, deque
import torch
agent = Agent(state_size, action_size, seed=0)

if torch.cuda.is_available():
    print("Using GPU")
else:
    print("Using CPU")

Using CPU


In [8]:
eps_start = 1.0
eps_end = 0.01
eps_decay=0.995
n_episodes = 2000
t_max = 300                        #  a limitation of the environment
eps = eps_start                    # initialize epsilon

scores = []                        # list containing scores from each episode
scores_window = deque(maxlen=100)  # last 100 scores

for i_episode in range(1,n_episodes+1):
    # action = np.random.randint(action_size)        # select an action
    score = 0                                        # initialize the score
    env_info = env.reset(train_mode=True)[brain_name]
    state = env_info.vector_observations[0]            # get the current state
    # reset environment
    for t in range(t_max):
        action = agent.act(state).astype(int)          # select an action
        # print(f"action selection:{action}")
        env_info = env.step(action)[brain_name]        # send the action to the environment
        next_state = env_info.vector_observations[0]   # get the next state
        reward = int(env_info.rewards[0])              # get the reward
        # if reward != 0.0:
            # print(reward, flush = True)
        done = env_info.local_done[0]                  # see if episode has finished
        agent.step(state, action, reward, next_state, done)

        score += reward                                # update the score
        state = next_state                             # roll over the state to next time step
        if done:                                       # exit loop if episode finished
            break

    scores_window.append(score)       # save most recent score
    scores.append(score)              # save most recent score ## DEBUG: score received was -5000
    eps = max(eps_end, eps_decay*eps) # decrease epsilon

    # print('\rEpisode {}\tAverage Score: {:.1f}'.format(i_episode, np.mean(scores_window)), end="", flush=True)
    if i_episode % 100 == 0:
        print('\rEpisode {}\tAverage Score: {:.2f}'.format(i_episode, np.mean(scores_window)))
    if np.mean(scores_window)>13:
        print('\nEnvironment solved in {:d} episodes!\tAverage Score: {:.2f}'.format(i_episode-100, np.mean(scores_window)))
        print(f"Last scores batch:\n{scores_window}")
        torch.save(agent.qnetwork_local.state_dict(), 'checkpoint.pth')
        break

Episode 100	Average Score: 0.47
Episode 200	Average Score: 5.83
Episode 300	Average Score: 7.77
Episode 400	Average Score: 8.98
Episode 500	Average Score: 12.31
Episode 600	Average Score: 11.80
Episode 700	Average Score: 11.75
Episode 800	Average Score: 12.30

Environment solved in 781 episodes!	Average Score: 13.10
Last scores batch:
deque([14, 12, 13, 14, 12, 14, 11, 13, 11, 10, 16, 14, 20, 14, 11, 12, 16, 15, 17, 11, 14, 13, 12, 12, 11, 9, 16, 17, 15, 15, 15, 12, 7, 15, 10, 12, 15, 15, 12, 5, 18, 9, 11, 16, 11, 11, 14, 12, 11, 11, 17, 10, 12, 17, 19, 18, 8, 10, 5, 18, 11, 10, 10, 11, 14, 14, 11, 14, 10, 12, 14, 11, 11, 12, 13, 11, 13, 16, 17, 17, 17, 17, 12, 12, 17, 9, 11, 17, 16, 14, 12, 17, 10, 15, 12, 13, 12, 15, 15, 15], maxlen=100)


In [12]:

#  env.close()

In [11]:

if torch.cuda.is_available():
    map_location=lambda storage, loc: storage.cuda()
    print("Using GPU")
else:
    map_location='cpu'
    print("Using CPU")

checkpoint = torch.load('checkpoint.pth', map_location=map_location)
agent.qnetwork_local.load_state_dict(checkpoint)


Using CPU


<All keys matched successfully>

##  Evaluation of the learned policy

In [12]:
# env_info = env.reset(train_mode=False)[brain_name] # reset the environment
env_info = env.reset(train_mode=False)[brain_name]
state = env_info.vector_observations[0]            # get the current state
score = 0                                          # initialize the score
i = 0
while True:

    action = agent.act(state).astype(int)          # select an action
    env_info = env.step(action)[brain_name]        # send the action to the environment
    next_state = env_info.vector_observations[0]   # get the next state
    reward = env_info.rewards[0]                   # get the reward
    done = env_info.local_done[0]                  # see if episode has finished

    score += reward                                # update the score
    state = next_state                             # roll over the state to next time step
    i+=1
    if done:                                       # exit loop if episode finished
        break
print(f"number of steps until finish the episode: {i}")
print("Score: {}".format(score))

number of steps until finish the episode: 300
Score: 16.0


In [13]:
env.close()